In [6]:
from utils.util import mkdir,seed_all
from omegaconf import OmegaConf
from cprint import *
from datasets.shape_net import ShapeNet
import torch
from models.Transform2D import Transform2D
from tqdm.notebook import tqdm
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup Expirement Folder

In [2]:
seed_all(111)
config = OmegaConf.load("./configs/global_configs.yaml")
cprint.ok(config)
description = "Testing Overfit" # Describe Experiment params here
logs_dir = config["logs_dir"]
mkdir(logs_dir)

experiment_dir = f"{logs_dir}/{config['name']}"
mkdir(experiment_dir)
mkdir(f"{experiment_dir}/checkpoints")
mkdir(f"{experiment_dir}/tb")
mkdir(f"{experiment_dir}/visuals")

{'logs_dir': 'logs', 'is_train': True, 'name': 'testingOverfit', 'device': 'cuda:0', 'batch_size': 8, 'n_epochs': 500, 'print_every': 300, 'validate_every': 300, 'model': {'lr': 0.0001}}
- logs directory found
- logs/testingOverfit directory found
- logs/testingOverfit/checkpoints directory found
- logs/testingOverfit/tb directory found
- logs/testingOverfit/visuals directory found


## Load Dataset & Dataloaders
This uses a random split for train/validation/test - might need to look into paper if they have a pre-defined split

In [3]:
dataset = ShapeNet(cat="airplane",is_overfit=False) #Change overfit param here & cat here
# train_ds, valid_ds, test_ds = torch.utils.data.random_split(
#     dataset, [64, 0, 0])
train_ds = dataset

train_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

validation_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

len(train_ds)


4045

## Loading Model

In [4]:
model = Transform2D()
# Declare device
device = torch.device('cpu')
if torch.cuda.is_available() and config['device'].startswith('cuda'):
    device = torch.device(config['device'])
    cprint.ok('Using device:', config['device'])
else:
    cprint.warn('Using CPU')

model.to(device)


Using device: cuda:0


Transform2D(
  (encoder): Encoder(
    (conv1a): ConvBlock(
      (conv): Conv2d(4, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (conv1b): ConvBlock(
      (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (conv2a): ConvBlock(
      (conv): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (conv2b): ConvBlock(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (res2): ResidualBlock(
      (conv1): ConvBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): LeakyReLU(negative_slope=0.01)
      )
      (conv2): ConvBlock(
        (conv): Conv2d(

## Training

Checklist:
- Add tensorboard and make sure it logs to the `${experiment_dir}/tb` folder
- Visualize some reconstructions on validation set and make sure it logs to the `${expirement_dir}/visuals` folders

In [5]:
train_loss_running = 0.
best_loss_val = np.inf
model.train()

for epoch in tqdm(range(config['n_epochs'])):
     for batch_idx, batch in tqdm(enumerate(train_dataloader)):
          ShapeNet.move_batch_to_device(batch, device)
          model.step(batch)
          train_loss_running += model.get_current_errors()
          iteration = epoch * len(train_dataloader) + batch_idx
          if iteration % config['print_every'] == (config['print_every'] - 1):
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {train_loss_running / config["print_every_n"]:.6f}')
            train_loss_running = 0.


     if iteration % config['validate_every'] == (config['validate_every'] - 1 or True):
          model.eval()
          loss_val = 0.
          for batch_val in validation_dataloader:
               ShapeNet.move_batch_to_device(batch_val, device)
               with torch.no_grad():
                   model.set_input(batch_val)
                   output = model.inference()
                   loss_val += model.get_current_errors()
          
          if loss_val < best_loss_val:
                    model.save()
                    best_loss_val = loss_val

          cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')
     model.update_lr()
         

  0%|          | 0/500 [00:00<?, ?it/s]

0it [00:00, ?it/s]

/home/youssef/.cache/pypoetry/virtualenvs/machine_learning_for_3d_geometry_final_pro-8w_t9tN0-py3.10/lib/python3.10/site-packages/torch/nn/functional.py:847: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool3d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


AttributeError: 'Transform2D' object has no attribute 'get_current_errors'

## Training on  single instance for checking

In [ ]:
train_loss_running = 0.
best_loss_val = np.inf
model.train()
data = dataset[0]
#data["images"] = data["images"].squeeze(0)
data["images"] = torch.tensor(data["images"]).cuda().float()
data["voxels"] = torch.tensor(data["voxels"]).cuda().float()
for epoch in tqdm(range(800)):
    model.step(data)
    loss = model.get_loss()
    cprint.ok(f'Model Loss: {loss["loss"]}')
    cprint.warn(f'Model Loss: {loss["loss_demo"]}')

    
    

In [ ]:
next(model.parameters()).device

In [ ]:
train_loss_running = 0.
best_loss_val = np.inf
model.train()

for epoch in tqdm(range(160)):
     for batch_idx, batch in tqdm(enumerate(train_dataloader)):
          ShapeNet.move_batch_to_device(batch, device)
          model.step(batch)
         #cprint.warn(f'Model Loss: {loss["loss_demo"]}')
     loss = model.get_loss()
     cprint.ok(f'Model Loss: {loss["loss"]}')
#      if(epoch%10==0):
#          torch.save(model.state_dict(), f"{experiment_dir}/checkpoints/epoch-{epoch}.ckpt")
         

In [ ]:
len(train_ds)
data  = train_ds[39]

data['voxels'].shape
data['images'].shape
#data= dataset[1]
data['images'].shape
data['voxels'].shape
data['images'] = data['images'][np.newaxis,:] 

In [ ]:
data['images'] = torch.tensor(data['images']).float().cuda()
data['voxels'] = torch.tensor(data['voxels']).float().cuda()

In [ ]:
from utils.visualizations import visualize_occupancy,visualize_images
data['voxels'].shape
tgt = data['voxels']

In [ ]:
visualize_occupancy(data["voxels"].squeeze(0).cpu(),flip_axes=True)

In [ ]:
x = model(data)

In [ ]:
# x.shape
sig = torch.nn.Sigmoid()
probs = sig(x)

# probs.shape
# probs.shape
# pred = probs[8]

# pred[pred>0.].shape
# pred[pred<0.3] = 0
# pred[pred>=0.1] = 1
# # pred.shape

# tgt[tgt!=0].shape
# pred[pred!=0].shape
# pred
# x.shape
# probs.shape

In [ ]:
probs.shape
pred = probs.clone()
pred[pred<0.4] = 0
pred[pred>=0.4] = 1


In [ ]:
visualize_occupancy(pred.detach().squeeze(0).cpu().numpy(),flip_axes=True)

In [ ]:
from utils.util import iou
tgt.shape
#pred.shape
#probs.max(dim=1)
iou(tgt.unsqueeze(0),probs.unsqueeze(0),0.6)

In [ ]:
probs.shape
#tgt.shape

In [ ]:
inter = probs.squeeze(0)

In [ ]:
inter.shape

In [ ]:
from einops import rearrange
inter = rearrange(inter, '(c1 c2 c3) -> c1 c2 c3',c1=32,c2=32,c3=32)

In [ ]:
inter[inter>0.4] = 1
inter[inter<=0.4] =0

In [ ]:
visualize_occupancy(inter.detach().squeeze(0).cpu().numpy(),flip_axes=True)

In [ ]:
loss2 = torch.nn.BCELoss()

In [ ]:
loss2(probs,tgt)

In [ ]:
probs.shape

In [ ]:
tgt.shape

In [ ]:
probs

In [ ]:
len(train_ds)

In [ ]:
from utils.util import iou
sig = torch.nn.Sigmoid()
with torch.no_grad():
    for i in range(10):
        data = dataset[i]
        data['images'] = torch.tensor(data['images']).unsqueeze(0).float().cuda()
        data['voxels'] = torch.tensor(data['voxels']).float().cuda()
        #import pdb;pdb.set_trace()
        tgt = data['voxels']
        pred = model(data)
        #pred = pred.squeeze(0)
        pred = sig(pred)
        #import pdb;pdb.set_trace()
        print(iou(tgt.unsqueeze(0),pred.unsqueeze(0),0.4))
        pred[pred>=0.4] = 1
        pred[pred<0.4] = 0
        #visualize_occupancy(tgt.detach().squeeze(0).cpu().numpy(),flip_axes=True)
        cprint.ok("=============================================================")
        #visualize_occupancy(pred.detach().squeeze(0).cpu().numpy(),flip_axes=True)
        #import pdb;pdb.set_trace()
    